In [1]:
import sys
import numpy as np
import import_ipynb
from scipy.sparse.linalg import spsolve
from pathlib import Path

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from fem_pre_processing import read_mesh
    from fem_processing import boundary_conditions, matrices_assembly
    from fem_pos_processing import graph_results
    print("Modules imports were successful!")
except ModuleNotFoundError as en:
    print(f"Modules were not found: {en}")
except ImportError as en:
    print(f"Error in import: {en}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Error in import: cannot import name 'graph_results' from 'pos_processing' (unknown location)


# Project 1: Coaxial Cable

Considere um cabo coaxial com dois dielétricos apresentado na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/coaxial_problem.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of Coaxial Problem.</figcaption>
</figure>

O raio do condutor interno é ``a`` e do condutor externo é ``b``. O primeiro dielétrico, $\varepsilon_{1}$, preenche o anel circular entre os raios ``a`` e ``c``, enquanto o segundo dielétrico, $\varepsilon_{2}$, preenche o anel entre os raios ``c`` e ``b``.  

Entre o condutor interno e o externo é aplicada uma diferença de potencial igual a $V$.  

Seja $V = 1$ (potencial zero em ``a`` e um em ``b``), $a = 2 \, mm$, $b = 8 \, mm$, $c = 5 \, mm$, $\varepsilon_{r1} = 2$ e $\varepsilon_{r2} = 4$. Para este problema, além da convergência nas normas $L_2$ e da energia, calcule a convergência para o valor da capacitância do dispositivo.

# 3-noded Linear Triangular Elements, $P_1$
## Pre-processor module
### Geometry and mesh Domain

In [2]:
FINITE_ELEMENT = ("Triangle", 1)

BOUNDARY = [{'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'inner_conductor'},
            {'tag': 102, 'type': 'Dirichlet', 'value': 1.0, 'name': 'outer_conductor'}]

MATERIAL = [{'tag': 201, 'name': 'medium_1', 
             'relative_electric_permittivity': 2, 'relative_magnetic_permeability': 1},
            {'tag': 202, 'name': 'medium_2',
             'relative_electric_permittivity': 4, 'relative_magnetic_permeability': 1}]

# Create mesh from file geometry
ps.create_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=1, view_mesh=False)

# Create mesh from file geometry
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY, MATERIAL, model='coaxial')

NameError: name 'ps' is not defined

## Mesh Data Dictionaries

In [3]:
cell_data = mesh_data['cell']; cell_data[1]

{'Tag': 25,
 'conn': [7, 25, 13],
 'material': {'tag': 201,
  'name': 'medium_1',
  'relative_electric_permittivity': 2,
  'relative_magnetic_permeability': 1}}

In [ ]:
nodes_data = mesh_data['nodes']; nodes_data[1]

{'xg': (0.002, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}}

In [5]:
conn = {key: cell['conn'] for key, cell in cell_data.items()}; conn[1]

[7, 25, 13]

## Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$

In [6]:
graph_results.plot_mesh(FINITE_ELEMENT, mesh_data, model='coaxial', Numbering=True)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pre_processing\pictures\coaxial_meshed_domain_Triangle1.svg


<figure>
    <img src="pre_processing/pictures/coaxial_meshed_domain_Triangle1.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$.</figcaption>
</figure>

## `apply_physics()`

In [7]:
mesh_data = ps.apply_physics(FINITE_ELEMENT, mesh_data); mesh_data['cell'][1]

{'Tag': 25,
 'conn': [7, 25, 13],
 'material': {'tag': 201,
  'name': 'medium_1',
  'relative_electric_permittivity': 2,
  'relative_magnetic_permeability': 1},
 'stiffness_a_value': 1.77083756256e-11,
 'mass_a_value': 1,
 'source': {'type': 'free_source', 'value': 0}}

## ``global_nodes_coordinates()``

In [8]:
nodes_coord = {key: value['xg'] for key, value in nodes_data.items()}; nodes_coord[1]

(0.002, 0.0)

## $x_g, y_g$ global coordinate

In [9]:
xg = {key: value['xg'][0] for key, value in nodes_data.items()}; xg[1]

0.002

## $a_e = (x_e, y_e)$ global element coordinate

In [10]:
ai = {key: [nodes_data[id]['xg'] for id in node_ids] for key, node_ids in conn.items()}
xi = {key: [coord[0] for coord in coords] for key, coords in ai.items()}
yi = {key: [coord[1] for coord in coords] for key, coords in ai.items()}
print("'a_e' global coordinates: "); ai[1]

'a_e' global coordinates: 


[(-0.001801937735804846, -0.0008677674782351008),
 (-0.003153391037658468, 1.23521800690691e-17),
 (-0.004504844339512103, -0.002169418695587776)]

In [11]:
print("'a_1' global coordinates: "); ai[1]

'a_1' global coordinates: 


[(-0.001801937735804846, -0.0008677674782351008),
 (-0.003153391037658468, 1.23521800690691e-17),
 (-0.004504844339512103, -0.002169418695587776)]

In [12]:
print("'Node 2' global coordinates: "); nodes_data[2]['xg']

'Node 2' global coordinates: 


(0.005, 0.0)

In [13]:
print("'x_1' x-coordinates: "); xi[1]

'x_1' x-coordinates: 


[-0.001801937735804846, -0.003153391037658468, -0.004504844339512103]

## `map_to_physical_coordinates()`

In [14]:
xi_master = (0, 1)
xg_1, yg_1 = matrices_assembly.isomapping_to_global_coordinates(ai[1], xi_master, FINITE_ELEMENT)
print(f"'e_1' Master coordinates: {xi_master} --> Global coordinates: ({xg_1}, {yg_1})")

'e_1' Master coordinates: (0, 1) --> Global coordinates: (-0.004504844339512103, -0.002169418695587776)


## Material Proprieties, $k_a$

In [15]:
ka = {key: value['stiffness_a_value'] for key, value in cell_data.items()}

## Source: Analytical source, $f(x)$

In [16]:
fx = {key: value['source']['value'] for key, value in cell_data.items()}

## Jacobian Matrix Transform

In [17]:
Je = matrices_assembly.jacobian(FINITE_ELEMENT, mesh_data, e=1, xik=(0, 0)) 
Jdet, Jinv = np.linalg.det(Je), np.linalg.inv(Je)
print("Jacobian matrix for element e_1:\n", Je)
print("Determinant of the Jacobian matrix for element e_1:", Jdet)

Jacobian matrix for element e_1:
 [[-0.00135145  0.00086777]
 [-0.00270291 -0.00130165]]
Determinant of the Jacobian matrix for element e_1: 4.104615282957144e-06


## Local Elements $e_1$

In [18]:
Ae, fe, Me, area_e = matrices_assembly.local_matrices(FINITE_ELEMENT, mesh_data, e=1)
print("Local stiffness matrix for element e_1:\n", Ae)
print("Local load vector for element e_1:\n", fe)
print("Area of element e_1: ", area_e)

Local stiffness matrix for element e_1:
 [[ 1.40921048e-11+0.j -1.39710359e-11+0.j -1.21068829e-13+0.j]
 [-1.39710359e-11+0.j  1.94141677e-11+0.j -5.44313178e-12+0.j]
 [-1.21068829e-13+0.j -5.44313178e-12+0.j  5.56420061e-12+0.j]]
Local load vector for element e_1:
 [[0.+0.j]
 [0.+0.j]
 [0.+0.j]]
Area of element e_1:  2.0523076209554954e-06


## Global “stiffness” matrix, $A_g$

In [ ]:
Ag, fg, Mg = matrices_assembly.global_matrix(FINITE_ELEMENT, mesh_data)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (26, 26)


## Imposition of Boundary Conditions
### $Dirichlet$ nodes

In [ ]:
dirichlet_nodes = {key: value 
                   for key, value in nodes_data.items() if value['bc']['type'] == 'Dirichlet'}
dirichlet_nodes

{1: {'xg': (0.002, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 3: {'xg': (0.008, 0.0), 'bc': {'type': 'Dirichlet', 'value': 1.0}},
 4: {'xg': (0.00124697960371747, 0.001563662964936058),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 5: {'xg': (-0.0004450418679126261, 0.001949855824363648),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 6: {'xg': (-0.001801937735804834, 0.0008677674782351253),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 7: {'xg': (-0.001801937735804846, -0.0008677674782351008),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 8: {'xg': (-0.0004450418679126447, -0.001949855824363644),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 9: {'xg': (0.00124697960371746, -0.001563662964936065),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 16: {'xg': (0.004987918414869878, 0.006254651859744231),
  'bc': {'type': 'Dirichlet', 'value': 1.0}},
 17: {'xg': (-0.001780167471650504, 0.007799423297454592),
  'bc': {'type': 'Dirichlet', 'value': 1.0}},
 18: {'xg': (-0.0072077509

In [21]:
Nn = len(nodes_data); Nd = len(dirichlet_nodes)
print(f"The entire domain has {Nn} nodes: {Nn - Nd} free nodes; {Nd} Dirichlet nodes.")

The entire domain has 26 nodes: 12 free nodes; 14 Dirichlet nodes.


In [22]:
free_nodes = {key: value for key, value in nodes_data.items() if value['bc']['type'] != 'Dirichlet'}

## Mapping Global nodes to reduced system

This code creates a dictionary called ``global_to_reduced``, which maps the global indices of the mesh nodes to the reduced indices, i.e. the indices that correspond only to the nodes that are not in the _Dirichlet boundary conditions_.

In [23]:
{global_id: idx + 1 for idx, global_id in enumerate(free_nodes.keys())}

{2: 1,
 10: 2,
 11: 3,
 12: 4,
 13: 5,
 14: 6,
 15: 7,
 22: 8,
 23: 9,
 24: 10,
 25: 11,
 26: 12}

## Processor Module
## Asymmetric Global matrix $A_g$ with boundary conditions

In [24]:
Ag, fg = boundary_conditions.apply_simple_dirichlet(Ag, fg, mesh_data)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (26, 26)


## Global Potential Vector

In [25]:
ug = spsolve(Ag.tocsr(), fg.toarray())
print("The solution vector u is: \n", ug)

The solution vector u is: 
 [0.        +0.j 0.78787879+0.j 1.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.78787879+0.j 0.78787879+0.j 0.78787879+0.j
 0.78787879+0.j 0.78787879+0.j 0.78787879+0.j 1.        +0.j
 1.        +0.j 1.        +0.j 1.        +0.j 1.        +0.j
 1.        +0.j 0.39393939+0.j 0.39393939+0.j 0.39393939+0.j
 0.39393939+0.j 0.39393939+0.j]


## Reduced Global matrix $A_{gr}$ with boundary conditions

In [26]:
Agr, fgr, Mgr = boundary_conditions.reduced_global_matrices(FINITE_ELEMENT, mesh_data)
print("Reduced global matrix shape:", Agr.shape)

Reduced global matrix shape: (12, 12)


## Global Potential Solution

In [27]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.global_potentials_solution(mesh_data, ur)
print("Reduced solution vector ur: ", ur)
print("Global potential vector V: "); potential_u

Reduced solution vector ur:  [0.78787879+0.j 0.78787879+0.j 0.78787879+0.j 0.78787879+0.j
 0.78787879+0.j 0.78787879+0.j 0.78787879+0.j 0.39393939+0.j
 0.39393939+0.j 0.39393939+0.j 0.39393939+0.j 0.39393939+0.j]
Global potential vector V: 


{1: 0.0,
 2: (0.7878787878787882+0j),
 3: 1.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: (0.7878787878787882+0j),
 11: (0.7878787878787877+0j),
 12: (0.787878787878788+0j),
 13: (0.7878787878787882+0j),
 14: (0.7878787878787878+0j),
 15: (0.7878787878787882+0j),
 16: 1.0,
 17: 1.0,
 18: 1.0,
 19: 1.0,
 20: 1.0,
 21: 1.0,
 22: (0.3939393939393942+0j),
 23: (0.39393939393939376+0j),
 24: (0.39393939393939414+0j),
 25: (0.3939393939393938+0j),
 26: (0.39393939393939376+0j)}

## Solution at each element

In [28]:
u_for_cell = {}
for en, cell in mesh_data['cell'].items():
    u_e = []
    for node in cell['conn']:
        u_e.append(potential_u[node])
    u_for_cell[en] = u_e

print("The complete solution for each cell."); u_for_cell[1]

The complete solution for each cell.


[0.0, (0.3939393939393938+0j), (0.7878787878787882+0j)]

## Interpolate Solution at element, $e_n$

In [29]:
xi_master = (0, 0.5)
en = 1
u_at_node = matrices_assembly.isomapping_to_global_coordinates(u_for_cell[en], xi_master, FINITE_ELEMENT)

print(f"Element e_{en}: Potential 'u' interpolate at xi = {xi_master} --> u(xi): {u_at_node}")

Element e_1: Potential 'u' interpolate at xi = (0, 0.5) --> u(xi): [0.39393939+0.j]


## Post-Processor

In [30]:
graph_results.fem_solution(FINITE_ELEMENT, mesh_data, potential_u, type='real')
ps.plot_analytical_solution()

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pos_processing\pictures\fem_solution_Triangle1.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pos_processing\pictures\analytical_solution.svg


### Figure 2: Scalar Field Distribution on a Triangular Mesh
<figure>
    <img src="pos_processing/pictures/fem_solution_Triangle1.svg" alt="Fig.2" style="width:100%;" />
    <figcaption>Figure 2: Post-Processor: Scalar Field Distribution.</figcaption>
</figure>

### Figure 4: Analytical Scalar Field Distribution on a Triangular Mesh
<figure>
    <img src="pos_processing/pictures/analytical_solution.svg" alt="Fig.4" style="width:100%;" />
    <figcaption>Figure 4: Analytical Scalar Field Distribution on a Triangular Mesh.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.